In [1]:
import PyPDF2
import pandas as pd
import fasttext
from io import StringIO
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk import sent_tokenize
from nltk.corpus import stopwords

In [2]:
nlp = en_core_web_sm.load()

pdf_file_reader = PyPDF2.PdfFileReader

file_path = "/Users/davidmpeletz/Desktop/islam.pdf"

# file_path = "/Users/davidmpeletz/Desktop/cso.pdf"

input1 = pdf_file_reader(file_path)

def get_all_content(input_pdf):

    if input_pdf.getNumPages() > 0:

        text_content = []
        str_content = ""
        stop_words = set(stopwords.words('english'))

        for i in range(input_pdf.getNumPages()):
            sentences = sent_tokenize(input_pdf.getPage(i).extractText())
            str_content += "".join(sentences)
            line_list = input_pdf.getPage(i).extractText().split('\n')
            words = re.split(r'\W+', input_pdf.getPage(i).extractText())
            new_str_content = [w for w in words if not w in stop_words]
            text_content.extend(words)
            doc = nlp(str_content)
#             print(str_content)
            
        return str_content

    else:
        raise Exception("The syllabus PDF must contain at least 1 page.")

In [3]:
get_all_content(input1)

' Reli 100.Introduction to Islam:\n Formation and Expansion\n             Fall 2018\n                       Professor James W. Laine\n OM 02\n, MWF 12:00\n-1:00\n             Old Main 103, x6789\n; laine@macalester.edu\n      Description:\n This will be an introductory course on Islam.We will study a variety of Islamic \ntraditions (ShariÕah law, Sufism, ShiÕi piety, kingship) \nin the classical period and early \nmodern world, with some attention given to the legacies of these tradit\nions in the \ncontemporary world.We will pay close attention to the quest for unity and a singular \nIslam in the face of the great variety of Muslim practices and perspectives in the world \npast and present.You should do the appropriate reading in advance of\n class.We will discuss \nreadings and sketch out critical issues for the topic at hand, both in terms of the subject \nmatter and debates about interpretation.In these sessions we will determine together the \nsorts of questions that will appear o

In [8]:
def contains_important_date(line):
    line = str(line).lower()
    doc = nlp(line)
    
    for ent in doc.ents:
        if('DATE' in ent.label_ or 
           'CARDINAL' in ent.label_ or 
           'TIME' in ent.label_):
            return 1
    return 0

def contains_grading_info(line):
    line = str(line).lower()
    doc = nlp(line)
    
    for ent in doc.ents:
        if(ent.label_ == 'PERCENT'):
            return 1
        
    return 0


def contains_email(line):
    line = str(line).lower()
    email_match = re.findall(r'[\w\.-]+@[\w\.-]+', line)
    if len(email_match) > 0:
        return 1
    return 0

In [9]:
def contains_useful_info(line):
    useful_info = ["quiz", "test", "exam", "final", "assessment", "due", 
                   "office hours", "room", "homework", "problem set", "essay", 
                   "paper", "reflection", "analysis", "project", "presentation", 
                   "percent", "%", "evaluation", "assessment", "grading", 
                   "grade", "policy", "report", "debate"]
    
    line = str(line).lower()
        
    for i in useful_info: 
        if i in line: 
            return 1
        
    return 0

In [10]:
def get_grading_policy(input_df):
    grading_df = input_df.loc[input_df['has_grading_info'] == 1]

    f_index = grading_df.head(1).index[0]
    l_index = grading_df.tail(1).index[0]
        
    num_categories = len(grading_df)   
    percentages = []
    descriptions = []
    
    string = ""
    for i in range(f_index, l_index + 2):
        string += line_df['line'][i]  

    values = string.split('%', num_categories)  

    for j in range(num_categories):
        percent_value = values[j][-2:] + "%"
        if j != num_categories - 1: 
            description = values[j+1][:-3]
        else: 
            description = values[j+1]

        percentages.append(percent_value)
        descriptions.append(description)

        return_df = pd.DataFrame()
        return_df['percentage'] = percentages
        return_df['description'] = descriptions
        
    return return_df
    
def get_email(input_df):
    email_frame = input_df.loc[input_df['has_email'] == 1]
    index = (email_frame.head(1).index[0])
    
    for i in (email_frame['line'][index].split()):
        email_match = re.findall(r'[\w\.-]+@[\w\.-]+', i)
        if len(email_match) > 0:
            return email_match[0]

In [11]:
return_bow = get_all_content(input1)

line_df = pd.DataFrame()
line_df['line'] = return_bow.split('\n')

In [12]:
line_df['has_useful_info'] = line_df['line'].apply(contains_useful_info)
line_df['has_email'] = line_df['line'].apply(contains_email)
line_df['has_grading_info'] = line_df['line'].apply(contains_grading_info)
line_df['has_date'] = line_df['line'].apply(contains_important_date)

In [13]:
len(line_df.loc[line_df['has_date']==1])

43

In [ ]:
def contains_office_hours(line):
    
    doc = nlp(line)

    for token in doc:
            
        if token.pos_ == 'NUM' and ':' in line:
            if 'MWF' in line: 
                return 1




In [ ]:
line_df['has_office_hours'] = line_df['line'].apply(contains_office_hours)

In [ ]:
def get_office_hours(input_df):
    office_hour_df = input_df.loc[input_df['has_office_hours'] == 1]
    index = office_hour_df.head(1).index[0]
    office_hour_df['has_office_hours'][index+1] = 1
    string = ""
    
    for j in range(index, index+2):
        string += input_df['line'][j]
    
    split_string = string.split(', ')[1]
    
    return split_string


In [ ]:
get_office_hours(line_df)

In [ ]:
get_grading_policy(line_df)